<a href="https://colab.research.google.com/github/ArmFriiz/Dicoding-Submission-FDL/blob/main/Analisis%20Sentimen/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load package dan library**

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 94.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import string
import nltk
import kagglehub
import gensim

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [3]:
path = kagglehub.dataset_download("bhimantoros/pretrained-word2vec-indonesia")
print("Path to dataset files:", path)

100%|██████████| 2.97G/2.97G [00:32<00:00, 99.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/bhimantoros/pretrained-word2vec-indonesia/versions/1


**Load dataset sentimen**

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/ArmFriiz/Dicoding-Submission-FDL/refs/heads/main/Analisis%20Sentimen/dataset_ulasan_playstore.csv')

In [5]:
df.head()

,content,score,label
0,CEPAT AMAN TERPERCAYA,5,Positif
1,top markotop,5,Positif
2,belanja online nomor 1 satu di Indonesia.,5,Positif
3,GANGGU DOANG LU ORG LAGI MAIN JUGA,1,Negatif
4,jadi langganan belanja di shopee..,5,Positif


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73478 entries, 0 to 73477
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  73478 non-null  object
 1   score    73478 non-null  int64 
 2   label    73478 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [7]:
df.describe(include='all')

,content,score,label
count,73478,73478.000000,73478
unique,72705,NaN,3
top,bagus,NaN,Positif
freq,5,NaN,51173
mean,NaN,3.862802,NaN
std,NaN,1.668418,NaN
min,NaN,1.000000,NaN
25%,NaN,2.000000,NaN
50%,NaN,5.000000,NaN
75%,NaN,5.000000,NaN


**Drop column score karena hanya digunakan untuk labelling awal saja**

In [8]:
df.drop(columns=['score'], inplace=True)

In [9]:
df.columns

Index(['content', 'label'], dtype='object')

**Cek ulang kondisi data**

In [10]:
df.isna().sum()

,0
content,0
label,0


In [11]:
df.duplicated().sum()

np.int64(472)

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.duplicated().sum()

np.int64(0)

In [14]:
df.describe(include='all')

,content,label
count,73006,73006
unique,72705,3
top,bagus dan cepat,Positif
freq,3,50760


**Load model normalisasi kata untuk perbaikan kata slank atau typo**

In [15]:
normalization_df = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/refs/heads/master/colloquial-indonesian-lexicon.csv')

In [16]:
normalization_df.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0


In [17]:
normalization_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15006 entries, 0 to 15005
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   slang          15006 non-null  object
 1   formal         15006 non-null  object
 2   In-dictionary  15006 non-null  int64 
 3   context        15006 non-null  object
 4   category1      15006 non-null  object
 5   category2      15006 non-null  object
 6   category3      15006 non-null  object
dtypes: int64(1), object(6)
memory usage: 820.8+ KB


In [18]:
normalization_dict = dict(zip(normalization_df['slang'], normalization_df['formal']))

**Cleaning Data (Case folding, Punctuation, Kata berulang, Slankword atau typo)**

In [19]:
def cleaning_text(text, normalization_dict):
  text = text.lower() # Ubah teks menjadi huruf kecil
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # Hapus mention
  text = re.sub(r'http\S+', '', text) # Hapus URL
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # Hapus hashtag
  text = re.sub(r'(.)\1{2,}', r'\1\1', text) # Hapus kata berulang yang muncul lebih dari 2x

  words = text.split()
  words = [normalization_dict.get(word, word) for word in words]
  text = ' '.join(words)

  text = ''.join([char for char in text if char.isalpha() or char.isspace()])
  text = ' '.join(text.split()) # Bersihkan spasi berlebih di tengah dan di ujung
  return text

In [20]:
df['clean_content'] = df['content'].apply(cleaning_text, normalization_dict=normalization_dict)

In [21]:
df.head()

,content,label,clean_content
0,CEPAT AMAN TERPERCAYA,Positif,cepat aman terpercaya
1,top markotop,Positif,top markotop
2,belanja online nomor 1 satu di Indonesia.,Positif,belanja online nomor satu di indonesia
3,GANGGU DOANG LU ORG LAGI MAIN JUGA,Negatif,ganggu doang lu orang lagi main juga
4,jadi langganan belanja di shopee..,Positif,jadi langganan belanja di shopee


**Encoding label data**

In [22]:
x = df['clean_content'].values

In [23]:
print(x)

['cepat aman terpercaya' 'top markotop'
 'belanja online nomor satu di indonesia' ...
 'berbelanja murah pengiriman cepat dan pelayanan yang ramah'
 'sejak shopee indonesia ada yang namanya outfit sepatu perlengkapan saya membuat kue semuanya saya pesan di shoopee apalagi kosmetik saya dan anak saya berlangganan di shoopee kalau ty pernah mengalami kekecewaan enggak karena shoope pernah tapi mereka siap terima keritikan dan masukan dari kita mungkin shallernya atau penjualnya yang harus jujur dan terbuka soalnya kadang kita meminta yang model atau warna apa kirimmya yang model dan warna lain tapi jarang sih pokoknya sukses ya shoope sudah membantu kami'
 'bagus banget buat belanja secara daring dan berguna untuk orang mageran']


In [24]:
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

In [25]:
df.head()

,content,label,clean_content,label_encoded
0,CEPAT AMAN TERPERCAYA,Positif,cepat aman terpercaya,2
1,top markotop,Positif,top markotop,2
2,belanja online nomor 1 satu di Indonesia.,Positif,belanja online nomor satu di indonesia,2
3,GANGGU DOANG LU ORG LAGI MAIN JUGA,Negatif,ganggu doang lu orang lagi main juga,0
4,jadi langganan belanja di shopee..,Positif,jadi langganan belanja di shopee,2


In [26]:
y = df['label_encoded'].values

In [27]:
print(y)
print(le.classes_)

[2 2 2 ... 2 2 2]
['Negatif' 'Netral' 'Positif']


In [28]:
y_onehot = to_categorical(y, num_classes=3)
y_onehot

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

**Splitting Data (Training-Testing-Validation)**

In [29]:
X_dev_raw, X_test_raw, y_dev, y_test = train_test_split(x, y_onehot, test_size=0.20, random_state=42, stratify=y)

# TAHAP 2: Pisahkan Data VALIDATION dari Data Development
# Dari 85% tadi, ambil sebagian untuk Validation (misal 15% dari total asli)
# Rumus: 0.15 / 0.85 ≈ 0.176
# X_train_raw, X_val_raw, y_train, y_val = train_test_split(
#     X_dev_raw, y_dev, test_size=0.176, random_state=42, stratify=y_dev
# )

# print(f"Jumlah Data Train (Belajar): {len(X_train_raw)}")
# print(f"Jumlah Data Val   (Try Out): {len(X_val_raw)}")
# print(f"Jumlah Data Test  (Ujian)  : {len(X_test_raw)}")

**Tokenization dan Padding**

In [30]:
EMBEDDING_DIM = 400

In [31]:
def process_tokenizer_dynamic(texts):
  # A. Inisialisasi Tokenizer TANPA batas num_words dulu
  tokenizer = Tokenizer(oov_token=OOV_TOK)
  tokenizer.fit_on_texts(texts)

  # B. Set MAX_VOCAB Sesuai Data
  # Ditambah 1 karena index 0 dipakai untuk padding
  vocab_size = len(tokenizer.word_index) + 1
  print(f"✅ Dynamic Vocab Size ditemukan: {vocab_size} kata unik.")

  # C. Convert ke Sequence untuk hitung panjang
  sequences = tokenizer.texts_to_sequences(texts)

  # D. Set MAX_LENGTH (Strategi: Percentile 95)
  # Kita cari panjang kalimat dimana 95% data tercakup.
  # Ini lebih aman daripada max() mutlak jika ada spammer yg nulis 2000 kata.
  lens = [len(s) for s in sequences]
  max_len_dynamic = int(np.percentile(lens, 95))

  # Opsi: Jika ingin benar-benar MAX mutlak (tanpa potong sama sekali):
  # max_len_dynamic = max(lens)

  print(f"✅ Dynamic Max Length (95th percentile): {max_len_dynamic} kata.")
  print(f"   (Kalimat terpanjang asli: {max(lens)} kata)")

  # E. Padding
  padded = pad_sequences(sequences, maxlen=max_len_dynamic, padding=PADDING_TYPE, truncating=TRUNC_TYPE)

  return padded, tokenizer, vocab_size, max_len_dynamic

**Load model Word2Vec untuk transfer learning (input model)**

In [32]:
w2v_model = gensim.models.Word2Vec.load("/root/.cache/kagglehub/datasets/bhimantoros/pretrained-word2vec-indonesia/versions/1/wiki.id.case.model").wv

In [33]:
def load_gensim_embedding(model, tokenizer, embedding_dim=400):
  word_index = tokenizer.word_index
  vocab_size = len(word_index) + 1
  embedding_matrix = np.zeros((vocab_size, embedding_dim))

  hits = 0
  misses = 0

  for word, i in word_index.items():
    if word in model:
      embedding_matrix[i] = model[word]
      hits += 1
    else:
      misses += 1
      # Word2Vec tidak bisa handle OOV, jadi biarkan 0 (atau random)

  print(f"Berhasil: {hits}, Gagal (OOV): {misses}")
  return embedding_matrix

# Cara Pakai di Benchmark:
# matrix = load_gensim_embedding(w2v_model, tokenizer, embedding_dim=200)